In [1]:
import numpy as np
import qutip as qt
import qutip.qip.operations as op
from scipy.linalg import expm

from PulseSequence import PulseSequence
from Pulse import Pulse
# from Operator import Operator
# from DensityMatrix import DensityMatrix

### Thermal density matrix

In [2]:
pho_th = qt.Qobj(np.diag([1, 0.6, -0.6, -1]), dims=([[2, 2], [2, 2]]))
pho_th

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.  -1. ]]

# OPERATORS

In [3]:
x = qt.sigmax()*0.5
y = qt.sigmay()*0.5
z = qt.sigmaz()*0.5
I = qt.qeye(2)

In [4]:
Ix = qt.tensor(x, I)
Iy = qt.tensor(y, I)
Iz = qt.tensor(z, I)

In [5]:
Sx = qt.tensor(I, x)
Sy = qt.tensor(I, y)
Sz = qt.tensor(I, z)

In [6]:
J = 215
tJ = 1/(2*J)
IzSz = qt.tensor(z, z)

In [7]:
Pulse("+2IzSz", IzSz, (2*np.pi*J)*(tJ) ).apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

In [8]:
# Delay for negative J coupling = 3*tJ; accurate upto global phase
Pulse("-2IzSz", IzSz, (2*np.pi*J)*(3*tJ) ).apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.70710678+0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678-0.70710678j]]

In [9]:
# IxSx = qt.tensor(x, x)
# IySy = qt.tensor(y, y)

# IxSy =  qt.tensor(x, y)
# IxSz =  qt.tensor(x, z)

# IySx =  qt.tensor(y, x)
# IySz =  qt.tensor(y, z)

# IzSx = qt.tensor(z, x)
# IzSy = qt.tensor(z, y)

We cant implement an $I_z/S_z$ pulse directly.  
Use $ R_z(\theta) = R_x(-\pi/2).R_y(\theta).R_x(\pi/2)$ 

In [10]:
def Iz_pulse(theta):
    return PulseSequence("Iz(theta)")\
        .add(Pulse("Ix(-pi/2)", Ix, -np.pi/2))\
        .add(Pulse(f"Iy({theta/np.pi}*pi)", Iy, theta))\
        .add(Pulse("Ix(pi/2)", Ix, np.pi/2))

In [11]:
Iz_pulse(np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]]

In [12]:
(-1j*np.pi/2*Iz).expm()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]]

Verified that $R_x(-\pi/2).R_y(\pi/2).R_x(\pi/2) = R_z(\pi/2)$ 

In [13]:
Iz_pulse(-np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678+0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678-0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

In [14]:
(+1j*np.pi/2*Iz).expm()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678+0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678-0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

Verified that $R_x(-\pi/2).R_y(-\pi/2).R_x(\pi/2) = R_z(-\pi/2)$ 

In [15]:
def Sz_pulse(theta):
    return PulseSequence("Sz(theta)")\
        .add(Pulse("Sx(-pi/2)", Sx, -np.pi/2))\
        .add(Pulse(f"Sy({theta/np.pi}*pi)", Sy, theta))\
        .add(Pulse("Sx(pi/2)", Sx, np.pi/2))

In [16]:
Sz_pulse(np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678-0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]]

In [17]:
Sz_pulse(np.pi/2).print_sequence()

Sz(theta) Pulse Sequence: Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2)

In [18]:
Sz_pulse(-np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678+0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

In [19]:
Sz_pulse(-np.pi/2).print_sequence()


Sz(theta) Pulse Sequence: Sx(-pi/2) -> Sy(-0.5*pi) -> Sx(pi/2)

# Q1: GATES

### PART A: One qubit X

In [20]:
X_i = Pulse("Ix(pi)", Ix, np.pi)
X_i.apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]]

In [21]:
X_s = Pulse("Sx(pi)", Sx, np.pi)
X_s.apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]]

### PART B: One qubit Hadamard

In [22]:
H_i = Pulse("Iy(-pi/2)", Iy, -np.pi/2)
H_i.apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [-0.70710678  0.          0.70710678  0.        ]
 [ 0.         -0.70710678  0.          0.70710678]]

In [23]:
H_i_dag = Pulse("Iy(+pi/2)", Iy, np.pi/2)
H_i_dag.apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678  0.         -0.70710678  0.        ]
 [ 0.          0.70710678  0.         -0.70710678]
 [ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]]

In [24]:
H_s = Pulse("Sy(-pi/2)", Sy, -np.pi/2)
H_s.apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678  0.70710678  0.          0.        ]
 [-0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.          0.         -0.70710678  0.70710678]]

In [25]:
H_s_dag = Pulse("Sy(+pi/2)", Sy, +np.pi/2)
H_s_dag.apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          0.70710678 -0.70710678]
 [ 0.          0.          0.70710678  0.70710678]]

### Part C: Two Qubit Controlled Z

In [26]:
cz_p = PulseSequence("CZ")\
    .add_seq(Iz_pulse(np.pi/2))\
    .add_seq(Sz_pulse(np.pi/2))\
    .add(Pulse("-2IzSz(aka delay(3/2J))", IzSz, 3*np.pi))

In [27]:
print(f"CZ multiplied by phase factor:")
np.exp(1j*np.pi/4)*cz_p.compile().get_operator()

CZ multiplied by phase factor:


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]]

In [28]:
cz_p.print_sequence()

CZ Pulse Sequence: Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J))

### Part D: Two Qubit CNOT

In [29]:
ncx = PulseSequence("Near CNOT")\
    .add(Pulse("Sx(pi/2)", Sx, np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, np.pi))\
    .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))

In [30]:
print(f"Near CX multiplied by phase factor: ")
np.exp(-1j*np.pi/4)*ncx.compile().get_operator()

Near CX multiplied by phase factor: 


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.-1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]]

In [31]:
ncx.print_sequence()

Near CNOT Pulse Sequence: Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [32]:
cx =  PulseSequence("CNOT")\
    .add(H_s)\
    .add_seq(cz_p)\
    .add(H_s_dag)

In [33]:
np.exp(1j*np.pi/4)*cx.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  0. -1.]
 [ 0.  0. -1.  0.]]

In [34]:
cx.print_sequence()

CNOT Pulse Sequence: Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2)

# (Q2) Oracles: Deutsch Jozsa

### f1(x) = 0 {Constant 0 function} 
f1(0)=0, f1(1)=0  
Do nothing

In [35]:
f1 = PulseSequence("f1")
f1.print_sequence()

f1 Pulse Sequence: 

### f2(x) = 1 {Constant 1 function} 
f2(0) = 1, f2(1) = 1  
Apply X gate on second qubit(C)


In [36]:
# f1(x) = 1 for x in {0,1}
# Apply X gate on second qubit(C)
f2 = PulseSequence("f4").add(X_s)
f2.print_sequence()

f4 Pulse Sequence: Sx(pi)

In [37]:
f2.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]]

### f3(x) = x xor y {Balanced function} 
f3(0) = 0, f3(1) = 1  
Apply CX gate, with control on $1^{st}$ qubit and target on $2^{nd}$ qubit.


In [38]:
f3 = PulseSequence("f4").add_seq(cx)
f3.print_sequence()

f4 Pulse Sequence: Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2)

In [39]:
f3.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.70710678+0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678+0.70710678j]
 [ 0.        +0.j          0.        +0.j         -0.70710678+0.70710678j
   0.        +0.j        ]]

### f4(x) = ~x xor y {Balanced function} 
f4(0) = 1, f4(1) = 0  
Apply anti-controlled CX gate, with control on $1^{st}$ qubit and target on $2^{nd}$ qubit. This can be done by applying an $X_{i}$ -> CX -> $X_{i}$ pulse sequence.


In [40]:
f4 = PulseSequence("f4")\
    .add(X_i)\
    .add_seq(cx)\
    .add(X_i)

In [41]:
f4.print_sequence()

f4 Pulse Sequence: Ix(pi) -> Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2) -> Ix(pi)

In [42]:
g = qt.basis(2,0)
e = qt.basis(2,1)

In [43]:
print("0-controlled CNOT upto global phase")
np.exp(1j*np.pi/4)*f4.compile().get_operator()

0-controlled CNOT upto global phase


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

#### Verification of 0-CNOT

In [44]:
np.exp(1j*np.pi/4)*f4.compile().get_operator()*qt.tensor(g,g)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]]

|00> -> |01>

In [45]:
np.exp(1j*np.pi/4)*f4.compile().get_operator()*qt.tensor(g,e)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]]

|01> -> |00>

In [46]:
np.exp(1j*np.pi/4)*f4.compile().get_operator()*qt.tensor(e,g)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]]

|10> -> |10>

In [47]:
np.exp(1j*np.pi/4)*f4.compile().get_operator()*qt.tensor(e,e)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [1.]]

|11> -> |11>

# (Q3) Deutsch Jozsa

### P1 and P2 operators for Temporal averaging  

In [48]:
# From lab manual
p1 =  PulseSequence("P1")\
    .add(Pulse("Sx(+pi/2)", Sx, +np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, +np.pi))\
    .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
    .add(Pulse("Ix(+pi/2)", Ix, +np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, +np.pi))\
    .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))

In [49]:
p1.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.-1.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]]

In [50]:
p1.print_sequence()

P1 Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2)

In [51]:
# From lab manual
p2 =  PulseSequence("P2")\
    .add(Pulse("Ix(+pi/2)", Ix, +np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, +np.pi))\
    .add(Pulse("Sx(+pi/2)", Sx, +np.pi/2))\
    .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, +np.pi))\
    .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))

In [52]:
p2.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.-1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]]

In [53]:
p2.print_sequence()

P2 Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2)

## Part(A) : |00> state
- A.1 : State Preparation
- A.2 : NCX gate
- A.3 : CX gate

### Part A.1 : State Preparation : |00>

#### Part A.1.1 : P0 simulation for |00>

In [54]:
p0_00 = PulseSequence("P0(00)")\
        .add(X_i)\
        .add(X_s)
p0_00.print_sequence()

P0(00) Pulse Sequence: Ix(pi) -> Sx(pi)

In [55]:
# %reset_selective p0_00

In [56]:
p0_00.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0. -1.]
 [ 0.  0. -1.  0.]
 [ 0. -1.  0.  0.]
 [-1.  0.  0.  0.]]

In [57]:
p0_rho00 = p0_00.compile().evolve_pho(pho_th)
p0_rho00.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.   0.6  0. ]
 [ 0.   0.   0.   1. ]]

In [58]:
p0_rho00.h_spectrum()
p0_rho00.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-1.6+0j)
Peak2 at 8.1836 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-0+0j)
Peak2 @ 80.1869 ppm with integral=(-0+0j)


In [59]:
# %reset_selective p0*00

#### Part A.1.2 : P1 simulation for |00>

In [60]:
p1_00 = PulseSequence("P1(00)")\
        .add_seq(p1)\
        .add(X_i)\
        .add(X_s)

In [61]:
p1_00.print_sequence()

P1(00) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Ix(pi) -> Sx(pi)

In [62]:
p1_00.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.+0.j  0.+1.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]]

In [63]:
p1_rho00 = p1_00.compile().evolve_pho(pho_th)
p1_rho00.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.  -0.6]]

In [64]:
p1_rho00.h_spectrum()
p1_rho00.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-2+0j)
Peak2 at 8.1836 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-2+0j)
Peak2 @ 80.1869 ppm with integral=(2+0j)


In [65]:
# %reset_selective p1_rho00


#### Part A.1.3 : P2 simulation for |00>

In [66]:
p2_00 = PulseSequence("P2(00)")\
        .add_seq(p2)\
        .add(X_i)\
        .add(X_s)

In [67]:
p2_00.print_sequence()

P2(00) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(pi) -> Sx(pi)

In [68]:
p2_00.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.+0.j  0.+1.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+1.j  0.+0.j  0.+0.j]]

In [69]:
p2_rho00 = p2_00.compile().evolve_pho(pho_th)
p2_rho00.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.   0.6]]

In [70]:
p2_rho00.h_spectrum()
p2_rho00.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-0.4+0j)
Peak2 at 8.1836 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-2+0j)
Peak2 @ 80.1869 ppm with integral=(-1+0j)


In [71]:
# %reset_selective p2_rho00

#### Part A.1.4 : Calculate temporal average for |00>

In [72]:
(p0_rho00.rho + p1_rho00.rho + p2_rho00.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.          0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.          0.33333333]]

### Part A.2: NCX gate simulation for |00>

#### Part A.2.1 : P0 NCX simulation for |00>

In [73]:
ncx_p0_00 = PulseSequence("NCX P0(00)")\
            .add_seq(p0_00)\
            .add_seq(ncx)
ncx_p0_00.print_sequence()

NCX P0(00) Pulse Sequence: Ix(pi) -> Sx(pi) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [74]:
np.exp(+1j*np.pi/4)*ncx_p0_00.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  0.-1.j  0.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+1.j  0.+0.j  0.+0.j]]

In [75]:
ncx_p0_rho00 = ncx_p0_00.compile().evolve_pho(pho_th)
ncx_p0_rho00.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.   0.6]]

In [76]:
ncx_p0_rho00.h_spectrum()
ncx_p0_rho00.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-2+0j)
Peak2 at 8.1836 ppm with integral=(-1.2+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-0+0j)
Peak2 @ 80.1869 ppm with integral=0j


In [77]:
# %reset_selective ncx_p*

#### Part A.2.2 : P1 NCX simulation for |00>

In [78]:
ncx_p1_00 = PulseSequence("NCX-P1(00)")\
            .add_seq(p1_00)\
            .add_seq(ncx)
ncx_p1_00.print_sequence()

NCX-P1(00) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Ix(pi) -> Sx(pi) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [79]:
np.exp(+1j*np.pi/4)*ncx_p1_00.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.+0.j 0.+0.j 0.+1.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]]

In [80]:
ncx_p1_rho00 = ncx_p1_00.compile().evolve_pho(pho_th)
ncx_p1_rho00.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.   1. ]]

In [81]:
ncx_p1_rho00.h_spectrum()
ncx_p1_rho00.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-0.4+0j)
Peak2 at 8.1836 ppm with integral=(-0.4+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-2+0j)
Peak2 @ 80.1869 ppm with integral=(-2+0j)


#### Part A.2.3 : P2 NCX simulation for |00>

In [82]:
ncx_p2_00 = PulseSequence("NCX-P2(00)")\
            .add_seq(p2_00)\
            .add_seq(ncx)
ncx_p2_00.print_sequence()

NCX-P2(00) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(pi) -> Sx(pi) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [83]:
np.exp(+1j*np.pi/4)*ncx_p2_00.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.+0.j 0.+0.j 0.+1.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]]

In [84]:
ncx_p2_rho00 = ncx_p2_00.compile().evolve_pho(pho_th)
ncx_p2_rho00.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   0.6  0. ]
 [ 0.   0.   0.  -0.6]]

In [85]:
ncx_p2_rho00.h_spectrum()
ncx_p2_rho00.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-1.6+0j)
Peak2 at 8.1836 ppm with integral=(1.6+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-2+0j)
Peak2 @ 80.1869 ppm with integral=(1+0j)


#### Part A.2.4 : Calculate temporal average for NCX(|00>)

In [86]:
(ncx_p0_rho00.rho + ncx_p1_rho00.rho + ncx_p2_rho00.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.          0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.          0.33333333]]

### Part A.3: CX gate simulation for |00> -- TODO

## Part(B) : |01> state
- B.1 : State Preparation
- B.2 : NCX gate
- B.3 : CX gate

### Part B.1 : State Preparation : |01>

#### Part B.1.1 : P0 simulation for |01>

In [87]:
p0_01 = PulseSequence("P0(00)")\
        .add(X_i)
p0_01.print_sequence()

P0(00) Pulse Sequence: Ix(pi)

In [88]:
p0_01.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]]

In [89]:
p0_rho01 = p0_01.compile().evolve_pho(pho_th)
p0_rho01.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.6  0.   0.   0. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.   0.6]]

In [90]:
p0_rho01.h_spectrum()
p0_rho01.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-1.6+0j)
Peak2 at 8.1836 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=0j
Peak2 @ 80.1869 ppm with integral=0j


#### Part B.1.2 : P1 simulation for |01>

In [91]:
p1_01 = PulseSequence("P1(01)")\
        .add_seq(p1)\
        .add(X_i)
p1_01.print_sequence()

P1(01) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Ix(pi)

In [92]:
p1_01.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.-1.j  0.+0.j  0.+0.j  0.+0.j]]

In [93]:
p1_rho01 = p1_01.compile().evolve_pho(pho_th)
p1_rho01.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.6  0.   0.   0. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.   1. ]]

In [94]:
p1_rho01.h_spectrum()
p1_rho01.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(1.2+0j)
Peak2 at 8.1836 ppm with integral=(-2+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(2+0j)
Peak2 @ 80.1869 ppm with integral=(-2+0j)


#### Part B.1.3 : P2 simulation for |01>

In [95]:
p2_01 = PulseSequence("P2(01)")\
        .add_seq(p2)\
        .add(X_i)
p2_01.print_sequence()

P2(01) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(pi)

In [96]:
p2_01.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.-1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]]

In [97]:
p2_rho01 = p2_01.compile().evolve_pho(pho_th)
p2_rho01.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.   0.6  0. ]
 [ 0.   0.   0.  -0.6]]

In [98]:
p2_rho01.h_spectrum()
p2_rho01.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(0.4+0j)
Peak2 at 8.1836 ppm with integral=(-0.4+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(2+0j)
Peak2 @ 80.1869 ppm with integral=(1+0j)


#### Part B.1.4 : Calculate temporal average for |01>

In [99]:
(p0_rho01.rho + p1_rho01.rho + p2_rho01.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.33333333  0.          0.          0.        ]
 [ 0.         -1.          0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.          0.33333333]]

In [100]:
# %reset_selective p2_*0*

### Part B.2 : Near CNOT(NCX) : |01>

#### Part B.2.1 : P0 simulation for NCX(|01>)

In [101]:
ncx_p0_01 = PulseSequence("NCX P0(01)")\
        .add_seq(p0_01)\
        .add_seq(ncx)

ncx_p0_01.print_sequence()

NCX P0(01) Pulse Sequence: Ix(pi) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [102]:
ncx_p0_01.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j          0.        +0.j         -0.70710678-0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.70710678-0.70710678j]
 [ 0.        +0.j         -0.70710678-0.70710678j  0.        +0.j
   0.        +0.j        ]
 [-0.70710678+0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]]

In [103]:
ncx_p0_rho01 = ncx_p0_01.compile().evolve_pho(pho_th)
ncx_p0_rho01.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.6  0.   0.   0. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.   0.6  0. ]
 [ 0.   0.   0.   1. ]]

In [104]:
ncx_p0_rho01.h_spectrum()
ncx_p0_rho01.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-1.2+0j)
Peak2 at 8.1836 ppm with integral=(-2+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=0j
Peak2 @ 80.1869 ppm with integral=(-0+0j)


#### Part B.2.2 : P1 simulation for NCX(|01>)

In [105]:
ncx_p1_01 = PulseSequence("NCX P1(01)")\
        .add_seq(p1_01)\
        .add_seq(ncx)
ncx_p1_01.print_sequence()

NCX P1(01) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Ix(pi) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [106]:
ncx_p1_01.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j          0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678-0.70710678j]
 [-0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.70710678+0.70710678j
   0.        +0.j        ]]

In [107]:
ncx_p1_rho01 = ncx_p1_01.compile().evolve_pho(pho_th)
ncx_p1_rho01.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.6  0.   0.   0. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.  -0.6]]

In [108]:
ncx_p1_rho01.h_spectrum()
ncx_p1_rho01.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-0.4+0j)
Peak2 at 8.1836 ppm with integral=(-0.4+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(2+0j)
Peak2 @ 80.1869 ppm with integral=(2+0j)


#### Part B.2.3 : P2 simulation for NCX(|01>)

In [109]:
ncx_p2_01 = PulseSequence("NCX P2(01)")\
        .add_seq(p2_01)\
        .add_seq(ncx)
ncx_p2_01.print_sequence()

NCX P2(01) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(pi) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [110]:
ncx_p2_01.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678-0.70710678j]
 [ 0.        +0.j          0.        +0.j          0.70710678+0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]]

In [111]:
ncx_p2_rho01 = ncx_p2_01.compile().evolve_pho(pho_th)
ncx_p2_rho01.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.   0.6]]

In [112]:
ncx_p2_rho01.h_spectrum()
ncx_p2_rho01.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(1.6+0j)
Peak2 at 8.1836 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(2+0j)
Peak2 @ 80.1869 ppm with integral=(-1+0j)


#### Part B.2.4 : Calculate temporal average for NCX(|01>)

In [113]:
(ncx_p0_rho01.rho + ncx_p1_rho01.rho + ncx_p2_rho01.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.33333333  0.          0.          0.        ]
 [ 0.         -1.          0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.          0.33333333]]

## Part(C) : |10> state
- C.1 : State Preparation
- C.2 : NCX gate
- C.3 : CX gate

### Part C.1 : State Preparation : |10>

#### Part C.1.1 : P0 simulation for |10>

In [114]:
p0_10 = PulseSequence("P0(10)")\
        .add(X_s)
p0_10.print_sequence()

P0(10) Pulse Sequence: Sx(pi)

In [115]:
p0_10.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]]

In [116]:
p0_rho10 = p0_10.compile().evolve_pho(pho_th)
p0_rho10.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.6  0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.  -0.6]]

In [117]:
p0_rho10.h_spectrum()
p0_rho10.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(1.6+0j)
Peak2 at 8.1836 ppm with integral=(1.6+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-0+0j)
Peak2 @ 80.1869 ppm with integral=(-0+0j)


#### Part C.1.2 : P1 simulation for |10>

In [118]:
p1_10 = PulseSequence("P1(10)")\
        .add_seq(p1)\
        .add(X_s)
p1_10.print_sequence()

P1(10) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Sx(pi)

In [119]:
p1_10.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.-1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+1.j  0.+0.j  0.+0.j]]

In [120]:
p1_rho10 = p1_10.compile().evolve_pho(pho_th)
p1_rho10.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.   0.6]]

In [121]:
p1_rho10.h_spectrum()
p1_rho10.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(2+0j)
Peak2 at 8.1836 ppm with integral=(-1.2+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(2+0j)
Peak2 @ 80.1869 ppm with integral=(-2+0j)


#### Part C.1.3 : P2 simulation for |10>

In [122]:
p2_10 = PulseSequence("P2(10)")\
        .add_seq(p2)\
        .add(X_s)
p2_10.print_sequence()

P2(10) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Sx(pi)

In [123]:
p2_10.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.-1.j  0.+0.j  0.+0.j  0.+0.j]]

In [124]:
p2_rho10 = p2_10.compile().evolve_pho(pho_th)
p2_rho10.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.6  0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.   1. ]]

In [125]:
p2_rho10.h_spectrum()
p2_rho10.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(0.4+0j)
Peak2 at 8.1836 ppm with integral=(-0.4+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-1+0j)
Peak2 @ 80.1869 ppm with integral=(-2+0j)


#### Part C.1.4 : Calculate temporal average for |10>

In [126]:
(p0_rho10.rho + p1_rho10.rho + p2_rho10.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.33333333  0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.         -1.          0.        ]
 [ 0.          0.          0.          0.33333333]]

### Part C.2 : Near CNOT(NCX) : |10>

#### Part C.2.1 : P0 simulation for NCX(|10>)

In [127]:
ncx_p0_10 = PulseSequence("NCX P0(10)")\
        .add_seq(p0_10)\
        .add_seq(ncx)

ncx_p0_10.print_sequence()

NCX P0(10) Pulse Sequence: Sx(pi) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [128]:
ncx_p0_10.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j         -0.70710678-0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.70710678-0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678+0.70710678j]]

In [129]:
ncx_p0_rho10 = ncx_p0_10.compile().evolve_pho(pho_th)
ncx_p0_rho10.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.6  0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.  -1. ]]

In [130]:
ncx_p0_rho10.h_spectrum()
ncx_p0_rho10.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(1.2+0j)
Peak2 at 8.1836 ppm with integral=(2+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-0+0j)
Peak2 @ 80.1869 ppm with integral=0j


#### Part C.2.2 : P1 simulation for NCX(|10>)

In [131]:
ncx_p1_10 = PulseSequence("NCX P1(10)")\
        .add_seq(p1_10)\
        .add_seq(ncx)
ncx_p1_10.print_sequence()

NCX P1(10) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Sx(pi) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [132]:
ncx_p1_10.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.70710678-0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.70710678+0.70710678j]]

In [133]:
ncx_p1_rho10 = ncx_p1_10.compile().evolve_pho(pho_th)
ncx_p1_rho10.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.   0.6  0. ]
 [ 0.   0.   0.  -1. ]]

In [134]:
ncx_p1_rho10.h_spectrum()
ncx_p1_rho10.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(0.4+0j)
Peak2 at 8.1836 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(2+0j)
Peak2 @ 80.1869 ppm with integral=(2+0j)


#### Part C.2.3 : P2 simulation for NCX(|10>)

In [135]:
ncx_p2_10 = PulseSequence("NCX P2(10)")\
        .add_seq(p2_10)\
        .add_seq(ncx)
ncx_p2_10.print_sequence()

NCX P2(10) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Sx(pi) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [136]:
ncx_p2_10.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j          0.        +0.j          0.70710678+0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j         -0.70710678-0.70710678j  0.        +0.j
   0.        +0.j        ]
 [-0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.70710678+0.70710678j]]

In [137]:
ncx_p2_rho10 = ncx_p2_10.compile().evolve_pho(pho_th)
ncx_p2_rho10.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.6  0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.  -1. ]]

In [138]:
ncx_p2_rho10.h_spectrum()
ncx_p2_rho10.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-1.6+0j)
Peak2 at 8.1836 ppm with integral=(1.6+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-1+0j)
Peak2 @ 80.1869 ppm with integral=(2+0j)


#### Part C.2.4 : Calculate temporal average for NCX(|10>)

In [139]:
(ncx_p0_rho10.rho + ncx_p1_rho10.rho + ncx_p2_rho10.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.33333333  0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.         -1.        ]]

## Part(D) : |11> state
- D.1 : State Preparation
- D.2 : NCX gate
- D.3 : CX gate

### Part D.1 : State Preparation : |11>

#### Part D.1.1 : P0 simulation for |11>

In [140]:
p0_11 = PulseSequence("P0(10)")
p0_11.print_sequence()

P0(10) Pulse Sequence: 

In [141]:
p0_11.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

In [142]:
p0_rho11 = p0_11.compile().evolve_pho(pho_th)
p0_rho11.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.  -1. ]]

In [143]:
p0_rho11.h_spectrum()
p0_rho11.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(1.6+0j)
Peak2 at 8.1836 ppm with integral=(1.6+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=0j
Peak2 @ 80.1869 ppm with integral=0j


#### Part D.1.2 : P1 simulation for |11>

In [144]:
p1_11 = PulseSequence("P1(11)")\
        .add_seq(p1)
p1_11.print_sequence()

P1(11) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2)

In [145]:
p1_11.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.-1.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]]

In [146]:
p1_rho11 = p1_11.compile().evolve_pho(pho_th)
p1_rho11.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.6  0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   0.6  0. ]
 [ 0.   0.   0.  -1. ]]

In [147]:
p1_rho11.h_spectrum()
p1_rho11.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-1.2+0j)
Peak2 at 8.1836 ppm with integral=(2+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-2+0j)
Peak2 @ 80.1869 ppm with integral=(2+0j)


#### Part D.1.3 : P2 simulation for |11>

In [148]:
p2_11 = PulseSequence("P2(11)")\
        .add_seq(p2)
p2_11.print_sequence()

P2(11) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2)

In [149]:
p2_11.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.-1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]]

In [150]:
p2_rho11 = p2_11.compile().evolve_pho(pho_th)
p2_rho11.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.6  0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.  -1. ]]

In [151]:
p2_rho11.h_spectrum()
p2_rho11.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(-0.4+0j)
Peak2 at 8.1836 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(1+0j)
Peak2 @ 80.1869 ppm with integral=(2+0j)


#### Part D.1.4 : Calculate temporal average for |11>

In [152]:
(p0_rho11.rho + p1_rho11.rho + p2_rho11.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.33333333  0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.         -1.        ]]

### Part D.2 : Near CNOT(NCX) : |11>

#### Part D.2.1 : P0 simulation for NCX(|11>)

In [153]:
ncx_p0_11 = PulseSequence("NCX P0(11)")\
        .add_seq(p0_11)\
        .add_seq(ncx)

ncx_p0_11.print_sequence()

NCX P0(11) Pulse Sequence: Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [154]:
ncx_p0_11.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.70710678-0.70710678j]
 [ 0.        +0.j          0.        +0.j         -0.70710678-0.70710678j
   0.        +0.j        ]]

In [155]:
ncx_p0_rho11 = ncx_p0_11.compile().evolve_pho(pho_th)
ncx_p0_rho11.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.  -0.6]]

In [156]:
ncx_p0_rho11.h_spectrum()
ncx_p0_rho11.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(2+0j)
Peak2 at 8.1836 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=0j
Peak2 @ 80.1869 ppm with integral=(-0+0j)


#### Part D.2.2 : P1 simulation for NCX(|11>)

In [157]:
ncx_p1_11 = PulseSequence("NCX P1(11)")\
        .add_seq(p1_11)\
        .add_seq(ncx)
ncx_p1_11.print_sequence()

NCX P1(11) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [158]:
ncx_p1_11.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j          0.        +0.j         -0.70710678-0.70710678j
   0.        +0.j        ]
 [ 0.70710678+0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678-0.70710678j]
 [ 0.        +0.j          0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]]

In [159]:
ncx_p1_rho11 = ncx_p1_11.compile().evolve_pho(pho_th)
ncx_p1_rho11.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.6  0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.   0.6]]

In [160]:
ncx_p1_rho11.h_spectrum()
ncx_p1_rho11.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(0.4+0j)
Peak2 at 8.1836 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(-2+0j)
Peak2 @ 80.1869 ppm with integral=(-2+0j)


#### Part D.2.3 : P2 simulation for |11>

In [161]:
ncx_p2_11 = PulseSequence("NCX P2(11)")\
        .add_seq(p2_11)\
        .add_seq(ncx)
ncx_p2_11.print_sequence()

NCX P2(11) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [162]:
ncx_p2_11.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j         -0.70710678-0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.70710678-0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678-0.70710678j]
 [-0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]]

In [163]:
ncx_p2_rho11 = ncx_p2_11.compile().evolve_pho(pho_th)
ncx_p2_rho11.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.6  0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.   1. ]]

In [164]:
ncx_p2_rho11.h_spectrum()
ncx_p2_rho11.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(1.6+0j)
Peak2 at 8.1836 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=(1+0j)
Peak2 @ 80.1869 ppm with integral=(-2+0j)


#### Part D.2.4 : Calculate temporal average for NCX(|11>)

In [165]:
(ncx_p0_rho10.rho + ncx_p1_rho10.rho + ncx_p2_rho10.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.33333333  0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.         -1.        ]]